<a href="https://colab.research.google.com/github/boneeyah/DS7337/blob/main/DeepVAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gluonts==0.12.6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.0 MB/s eta 0:00:00


In [3]:
!pip install mxnet==1.6
!pip install orjson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 MB 5.8 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import numpy as np
from gluonts.mx.trainer import Trainer
from gluonts.mx.model.deepvar import DeepVAREstimator
from gluonts.dataset.common import ListDataset

In [16]:
df_50 = np.loadtxt("https://raw.githubusercontent.com/boneeyah/DS7337/main/nonlinear_synthetic_50.csv",delimiter=",",skiprows=1,usecols=(0,4,5))

h_list = [[3,5,8],[5,10,25],[50,100,200]]
freq_list = ["M","W","D"]

In [17]:
df_50 = df_50[df_50[:,0]==1]

In [18]:
def model_fit(data_list,horizon_list,freq_list):
  a_preds = {}
  for data in data_list:
    labels = list(set(data[:,0]))
    freq = freq_list[data_list.index(data)]
    for series in labels:
      df = data[data[:,0]==series]
      df = df[:,1:3]
      for h in horizon_list[data_list.index(data)]:
        train = df[:-h]
        test = df[-h:]
        train = ListDataset([{"start":"2000-01-01","target":train.T}],one_dim_target=False,freq=freq)
        estimator = DeepVAREstimator(freq=freq, prediction_length=h,target_dim=2,scaling=True)
        predictor = estimator.train(training_data=train)
        preds = predictor.predict(train)
        preds = list(preds)[0].quantile(0.5)
        a_preds.update({'Data{}{}_h{}'.format(data_list.index(data),series,h):preds})
  return(a_preds)

In [20]:
model_fit([df_50],[[3,5]],["M"])

100%|██████████| 50/50 [00:02<00:00, 19.47it/s, epoch=100/100, avg_epoch_loss=0.819]


{'Data01.0_h3': array([[-9.912015 , 13.744956 ],
        [-5.985606 , 12.929554 ],
        [-3.2204964, 11.151989 ]], dtype=float32),
 'Data01.0_h5': array([[ 0.06019839,  4.2409544 ],
        [-2.6822908 ,  3.1797833 ],
        [-9.088064  ,  4.399542  ],
        [-6.9052353 ,  8.219521  ],
        [-3.274787  ,  7.8054113 ]], dtype=float32)}

In [11]:
df = df_50[df_50[:,0]==1]
df = df[:,1:3]

train = df[:-5]
test = df[-5:]
train = ListDataset([{"start":"2000-01-01","target":train.T}],one_dim_target=False,freq="M")
estimator = DeepVAREstimator(freq="M", prediction_length=5,target_dim=2,scaling=True)
predictor = estimator.train(training_data=train,num_workers = 4)
preds = predictor.predict(train)

100%|██████████| 50/50 [00:01<00:00, 30.61it/s, epoch=100/100, avg_epoch_loss=0.306]
